In [51]:
# 读取数据
import time
import pandas as pd #数据分析
import matplotlib.pyplot as plt
import numpy as np
#混淆矩阵
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    import itertools
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    # plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
time_start=time.time()
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
data= pd.read_csv("E:\Pycharm\Intrusion_Detection\kddcup.data_10_percent.csv",  header=None,names = col_names)

#去重
# import matplotlib.pyplot as plt
# IsDuplicated=data.duplicated()
#
# IsDuplicated.value_counts().plot(kind='bar')
# plt.show()
data_1=data.drop_duplicates()

#one-hot
dummies_protocol = pd.get_dummies(data_1["protocol_type"], prefix='protocol')
dummies_flag = pd.get_dummies(data_1["flag"], prefix='flag')
# dummies_service = pd.get_dummies(data_1["service"], prefix='service')
# data_2 = pd.concat([data_1, dummies_protocol,dummies_flag,dummies_service], axis=1)
data_2 = pd.concat([data_1, dummies_protocol,dummies_flag], axis=1)
# data_2
#特征选择(by "A feature reduced intrusion detection system using ANN classifier",25个特征）
#建立X,y
feature_selection=["duration",
    "dst_bytes","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_access_files",
    "num_outbound_cmds","is_guest_login","rerror_rate","srv_diff_host_rate",
    "dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate",
    "protocol_icmp","protocol_tcp","protocol_udp",
    "flag_OTH","flag_REJ","flag_RSTO","flag_RSTOS0","flag_RSTR",
    "flag_S0","flag_S1","flag_S2","flag_S3","flag_SF","flag_SH"]
X_3=data_2[feature_selection]
y_3=data_2['label'].copy()   #一维
##y的处理
u2r=["buffer_overflow.","loadmodule.","perl.","rootkit."]
r2l=["ftp_write.","imap.","guess_passwd.","phf.","spy.","multihop.","warezmaster.","warezclient."]
dos=["back.","land.","pod.","neptune.","smurf.","teardrop."]
probe=["satan.","portsweep.","ipsweep.","nmap."]
for i in u2r:
    y_3[y_3==i]='1' #u2r
for i in r2l:
    y_3[y_3==i]='1' #r2l
for i in dos:
    y_3[y_3==i]='1'  #dos
    
for i in probe:
    y_3[y_3==i]='1' #probe
y_3[y_3=="normal."]='0' #normal
y_3=np.array(y_3)  #变成array格式，一维

In [52]:
from collections import Counter



import numpy as np

from sklearn.base import is_regressor

from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble.forest import BaseForest

from sklearn.neighbors import NearestNeighbors

from sklearn.preprocessing import normalize

from sklearn.tree.tree import BaseDecisionTree

from sklearn.utils import check_random_state

from sklearn.utils import check_array

#from sklearn.utils import shuffle
from sklearn.utils import check_X_y




class SMOTE(object):

    """Implementation of Synthetic Minority Over-Sampling Technique (SMOTE).



    SMOTE performs oversampling of the minority class by picking target 

    minority class samples and their nearest minority class neighbors and 

    generating new samples that linearly combine features of each target 

    sample with features of its selected minority class neighbors [1].



    Parameters

    ----------

    k_neighbors : int, optional (default=5)

        Number of nearest neighbors.

    random_state : int or None, optional (default=None)

        If int, random_state is the seed used by the random number generator.

        If None, the random number generator is the RandomState instance used

        by np.random.



    References

    ----------

    .. [1] N. V. Chawla, K. W. Bowyer, L. O. Hall, and P. Kegelmeyer. "SMOTE:

           Synthetic Minority Over-Sampling Technique." Journal of Artificial

           Intelligence Research (JAIR), 2002.

    """



    def __init__(self, k_neighbors=5, random_state=None):

        self.k = k_neighbors

        self.random_state = random_state



    def sample(self, n_samples):

        """Generate samples.



        Parameters

        ----------

        n_samples : int

            Number of new synthetic samples.



        Returns

        -------

        S : array, shape = [n_samples, n_features]

            Returns synthetic samples.

        """

        np.random.seed(seed=self.random_state)



        S = np.zeros(shape=(n_samples, self.n_features))

        # Calculate synthetic samples.

        for i in range(n_samples):

            j = np.random.randint(0, self.X.shape[0])



            # Find the NN for each sample.

            # Exclude the sample itself.

            nn = self.neigh.kneighbors(self.X[j].reshape(1, -1),

                                       return_distance=False)[:, 1:]

            nn_index = np.random.choice(nn[0])



            dif = self.X[nn_index] - self.X[j]

            gap = np.random.random()



            S[i, :] = self.X[j, :] + gap * dif[:]



        return S



    def fit(self, X):

        """Train model based on input data.



        Parameters

        ----------

        X : array-like, shape = [n_minority_samples, n_features]

            Holds the minority samples.

        """

        self.X = X

        self.n_minority_samples, self.n_features = self.X.shape



        # Learn nearest neighbors.

        self.neigh = NearestNeighbors(n_neighbors=self.k + 1)

        self.neigh.fit(self.X)



        return self





class SMOTEBoost(AdaBoostClassifier):

    """Implementation of SMOTEBoost.



    SMOTEBoost introduces data sampling into the AdaBoost algorithm by

    oversampling the minority class using SMOTE on each boosting iteration [1].



    This implementation inherits methods from the scikit-learn 

    AdaBoostClassifier class, only modifying the `fit` method.



    Parameters

    ----------

    n_samples : int, optional (default=100)

        Number of new synthetic samples per boosting step.

    k_neighbors : int, optional (default=5)

        Number of nearest neighbors.

    base_estimator : object, optional (default=DecisionTreeClassifier)

        The base estimator from which the boosted ensemble is built.

        Support for sample weighting is required, as well as proper `classes_`

        and `n_classes_` attributes.

    n_estimators : int, optional (default=50)

        The maximum number of estimators at which boosting is terminated.

        In case of perfect fit, the learning procedure is stopped early.

    learning_rate : float, optional (default=1.)

        Learning rate shrinks the contribution of each classifier by

        ``learning_rate``. There is a trade-off between ``learning_rate`` and

        ``n_estimators``.

    algorithm : {'SAMME', 'SAMME.R'}, optional (default='SAMME.R')

        If 'SAMME.R' then use the SAMME.R real boosting algorithm.

        ``base_estimator`` must support calculation of class probabilities.

        If 'SAMME' then use the SAMME discrete boosting algorithm.

        The SAMME.R algorithm typically converges faster than SAMME,

        achieving a lower test error with fewer boosting iterations.

    random_state : int or None, optional (default=None)

        If int, random_state is the seed used by the random number generator.

        If None, the random number generator is the RandomState instance used

        by np.random.



    References

    ----------

    .. [1] N. V. Chawla, A. Lazarevic, L. O. Hall, and K. W. Bowyer.

           "SMOTEBoost: Improving Prediction of the Minority Class in

           Boosting." European Conference on Principles of Data Mining and

           Knowledge Discovery (PKDD), 2003.

    """



    def __init__(self,

                 n_samples=100,

                 k_neighbors=5,

                 base_estimator=None,

                 n_estimators=50,

                 learning_rate=1.,

                 algorithm='SAMME.R',

                 random_state=None):



        self.n_samples = n_samples

        self.algorithm = algorithm

        self.smote = SMOTE(k_neighbors=k_neighbors,

                           random_state=random_state)



        super(SMOTEBoost, self).__init__(

            base_estimator=base_estimator,

            n_estimators=n_estimators,

            learning_rate=learning_rate,

            random_state=random_state)



    def fit(self, X, y, sample_weight=None, minority_target=None):

        """Build a boosted classifier/regressor from the training set (X, y),

        performing SMOTE during each boosting step.



        Parameters

        ----------

        X : {array-like, sparse matrix} of shape = [n_samples, n_features]

            The training input samples. Sparse matrix can be CSC, CSR, COO,

            DOK, or LIL. COO, DOK, and LIL are converted to CSR. The dtype is

            forced to DTYPE from tree._tree if the base classifier of this

            ensemble weighted boosting classifier is a tree or forest.

        y : array-like of shape = [n_samples]

            The target values (class labels in classification, real numbers in

            regression).

        sample_weight : array-like of shape = [n_samples], optional

            Sample weights. If None, the sample weights are initialized to

            1 / n_samples.

        minority_target : int

            Minority class label.



        Returns

        -------

        self : object

            Returns self.



        Notes

        -----

        Based on the scikit-learn v0.18 AdaBoostClassifier and

        BaseWeightBoosting `fit` methods.

        """

        # Check that algorithm is supported.

        if self.algorithm not in ('SAMME', 'SAMME.R'):

            raise ValueError("algorithm %s is not supported" % self.algorithm)



        # Check parameters.

        if self.learning_rate <= 0:

            raise ValueError("learning_rate must be greater than zero")



        if (self.base_estimator is None or

                isinstance(self.base_estimator, (BaseDecisionTree,

                                                 BaseForest))):

            DTYPE = np.float64  # from fast_dict.pxd

            dtype = DTYPE

            accept_sparse = 'csc'

        else:

            dtype = None

            accept_sparse = ['csr', 'csc']



        X, y = check_X_y(X, y, accept_sparse=accept_sparse, dtype=dtype,

                         y_numeric=is_regressor(self))



        if sample_weight is None:

            # Initialize weights to 1 / n_samples.

            sample_weight = np.empty(X.shape[0], dtype=np.float64)

            sample_weight[:] = 1. / X.shape[0]

        else:

            sample_weight = check_array(sample_weight, ensure_2d=False)

            # Normalize existing weights.

            sample_weight = sample_weight/sample_weight.sum(dtype=np.float64)



            # Check that the sample weights sum is positive.

            if sample_weight.sum() <= 0:

                raise ValueError(

                    "Attempting to fit with a non-positive "

                    "weighted number of samples.")



        if minority_target is None:

            # Determine the minority class label.

            stats_c_ = Counter(y)

            maj_c_ = max(stats_c_, key=stats_c_.get)

            min_c_ = min(stats_c_, key=stats_c_.get)

            self.minority_target = min_c_

        else:

            self.minority_target = minority_target



        # Check parameters.

        self._validate_estimator()



        # Clear any previous fit results.

        self.estimators_ = []

        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float64)

        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float64)



        random_state = check_random_state(self.random_state)



        for iboost in range(self.n_estimators):

            # SMOTE step.

            X_min = X[np.where(y == self.minority_target)]

            self.smote.fit(X_min)

            X_syn = self.smote.sample(self.n_samples)

            y_syn = np.full(X_syn.shape[0], fill_value=self.minority_target,

                            dtype=np.int64)



            # Normalize synthetic sample weights based on current training set.

            sample_weight_syn = np.empty(X_syn.shape[0], dtype=np.float64)

            sample_weight_syn[:] = 1. / X.shape[0]



            # Combine the original and synthetic samples.

            X = np.vstack((X, X_syn))

            y = np.append(y, y_syn)



            # Combine the weights.

            sample_weight = \
                np.append(sample_weight, sample_weight_syn).reshape(-1, 1)

            sample_weight = \
                np.squeeze(normalize(sample_weight, axis=0, norm='l1'))



            # X, y, sample_weight = shuffle(X, y, sample_weight,

            #                              random_state=random_state)



            # Boosting step.

            sample_weight, estimator_weight, estimator_error = self._boost(

                iboost,

                X, y,

                sample_weight,

                random_state)



            # Early termination.

            if sample_weight is None:

                break



            self.estimator_weights_[iboost] = estimator_weight

            self.estimator_errors_[iboost] = estimator_error



            # Stop if error is zero.

            if estimator_error == 0:

                break



            sample_weight_sum = np.sum(sample_weight)



            # Stop if the sum of sample weights has become non-positive.

            if sample_weight_sum <= 0:

                break



            if iboost < self.n_estimators - 1:

                # Normalize.

                sample_weight /= sample_weight_sum



        return self

In [53]:
oversampler=SMOTEBoost()
print(oversampler)

SMOTEBoost(algorithm='SAMME.R', base_estimator=None, k_neighbors=None,
      learning_rate=1.0, n_estimators=50, n_samples=100, random_state=None)


In [54]:
oversampler.fit(X_3,y_3)

TypeError: unorderable types: int() > str()

In [7]:

#标准化
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(X_smote)
X_s=scaler.transform(X_smote)  #X是ndarray

#可视化
y_smote=pd.DataFrame(y_s,columns=['label'])
print(y_smote.label.value_counts())

classes_1=['Probing','Normal','U2R','DOS','R2L']
plt.figure()
plt.title("smoteBoost")
y_smote['label'].value_counts().plot(kind='bar',rot=45)
# y_sc=[87832,87832,87832,87832,87832]
# plt.pie(y_sc, colors=colors_1, labels=classes_1,autopct='%1.2f%%',pctdistance=0.7, shadow=True)
from sklearn.model_selection import train_test_split
X_train_1,X_test_1,y_train_1,y_test_1=train_test_split(X_s,y_s,test_size=0.2,random_state=0)  #切分样本
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(X_test_1)
plt.figure()
plt.title("smoteBoost")
classes=['Normal','Probing','DOS','U2R','R2L']
colors=['blue','red','y','m','w']
for index,label,color in zip(range(len(classes)),classes,colors):
    plt.scatter(X_embedded[y_test_1==label,0],
                X_embedded[y_test_1==label,1],
                label=classes[index],
                c=color)
plt.legend(loc='best')
# plt.show()

#分类器
from sklearn.ensemble import RandomForestClassifier
##建立模型
clf_2 = RandomForestClassifier(n_estimators=100)
#验证测试样本
clf_2.fit(X_train_1,y_train_1)
preditions_smote=clf_2.predict(X_test_1)
#学习曲线
import numpy as np
from sklearn.model_selection import learning_curve
train_sizes,train_scores,test_scores=learning_curve(estimator=clf_2,
                                X=X_train_1,y=y_train_1,
                                train_sizes=np.linspace(0.05,1,10),
                                 cv=10, n_jobs=1,random_state=0)
train_mean_1=np.mean(train_scores,axis=1)
test_mean_1=np.mean(test_scores,axis=1)
train_std_1=np.std(train_scores,axis=1)
test_std_1=np.std(train_scores,axis=1)

TypeError: 'bool' object is not iterable